# Analysis of played music in main Polish broadcasts

Let's insert scraped data from www.radiospis.pl by radiospis_crawler to spark.

In [3]:
dbutils.fs.ls("dbfs:/FileStore/tables/")

Out[15]: [FileInfo(path='dbfs:/FileStore/tables/songs.csv', name='songs.csv', size=1781046),
 FileInfo(path='dbfs:/FileStore/tables/songs_20200317.csv', name='songs_20200317.csv', size=1712264),
 FileInfo(path='dbfs:/FileStore/tables/songs_20200321.csv', name='songs_20200321.csv', size=1748376)]

In [4]:
songs = sqlContext.read\
    .format("com.databricks.spark.csv")\
    .option('header', 'true')\
    .option('mode', 'DROPMALFORMED')\
    .load('dbfs:/FileStore/tables/songs*.csv')

In [5]:
print(songs.count())

96212

Create temporary view for data from spark

In [7]:
spark.catalog.dropTempView("songs")
songs.createOrReplaceTempView("songs")

Enrich data columns with time. author and title

In [9]:
from pyspark.sql.functions import split, col, date_format
spark.conf.set("spark.sql.session.timeZone", "Europe/Warsaw")
songs = spark.sql("""
select distinct
  broadcast,
  song_name,
  to_timestamp(datetime,'dd-MM-yy HH:mm') as datetime,
  year(to_timestamp(datetime,'dd-MM-yy HH:mm')) as year,
  month(to_timestamp(datetime,'dd-MM-yy HH:mm')) as month,
  day(to_timestamp(datetime,'dd-MM-yy HH:mm')) as day,
  hour(to_timestamp(datetime,'dd-MM-yy HH:mm')) as hour,
  minute(to_timestamp(datetime,'dd-MM-yy HH:mm')) as minute
from songs
where song_name not in ('Radiozet - Radio zet i juz', 'Vox Fm')
  and broadcast not in ('broadcast')
  and song_name not like '%Vox Fm%'
""")

songs=songs.withColumn('week_day', date_format(col('datetime'), 'E'))
songs=songs.withColumn('author', split(col('song_name'),' - ').getItem(0))
songs=songs.withColumn('title', split(col('song_name'),' - ').getItem(1))
songs=songs.withColumn('datetime', date_format(col('datetime'), 'yyyy-MM-dd'))
spark.catalog.dropTempView("songs")
songs.createOrReplaceTempView("songs")

Let's check the sample of data

In [11]:
display(songs.limit(20))

broadcast,song_name,datetime,year,month,day,hour,minute,week_day,author,title
rmf-fm,Elize - Automatic,2020-03-12,2020,3,12,6,12,Thu,Elize,Automatic
rmf-fm,Big Mountain - Baby i love your way,2020-03-10,2020,3,10,9,15,Tue,Big Mountain,Baby i love your way
rmf-fm,Kelly Clarkson - Behind these hazel eyes,2020-03-09,2020,3,9,11,5,Mon,Kelly Clarkson,Behind these hazel eyes
rmf-fm,Walk The Moon - Shut up and dance,2020-03-09,2020,3,9,11,51,Mon,Walk The Moon,Shut up and dance
rmf-fm,Golec Uorkiestra - Nie gaś ducha,2020-03-13,2020,3,13,19,30,Fri,Golec Uorkiestra,Nie gaś ducha
rmf-fm,Margaret - In my cabana,2020-03-07,2020,3,7,18,21,Sat,Margaret,In my cabana
rmf-fm,Jubel - Blue jeans,2020-03-09,2020,3,9,19,49,Mon,Jubel,Blue jeans
rmf-fm,Komodo / Michael Shynes - Is this love,2020-03-13,2020,3,13,20,12,Fri,Komodo / Michael Shynes,Is this love
rmf-fm,Michael Jackson - They don't care about us,2020-03-11,2020,3,11,21,6,Wed,Michael Jackson,They don't care about us
zet,Lemon - Jutro,2020-03-08,2020,3,8,2,15,Sun,Lemon,Jutro


Number of unique rows in view

In [13]:
print(songs.count())

31309

Analysis data range is following:

In [15]:
from pyspark.sql.functions import min, max, date_format, col
display(songs\
        .agg(min('datetime'), max('datetime'))\
        .withColumnRenamed('min(datetime)', 'date_from')\
        .withColumnRenamed('max(datetime)', 'date_to')
)

date_from,date_to
2020-03-07,2020-03-21


Most often played songs in broadcasts

In [17]:
display(songs.groupBy('song_name').count().orderBy("count", ascending=False).limit(20))

song_name,count
Sanah - Szampan,213
Ava Max - Salt,206
Viki Gabor / Kayah - Ramie w ramie,191
The Weeknd - Blinding lights,184
Alan Walker/ava Max - Alone pt. ii,170
Felix Jaehn‚ Vize‚ Miss Li - Close your eyes,160
Topic‚ A7S - Breaking me,156
Dua Lipa - Physical,146
Patryk Skoczyński - Dom,126
Lewis Capaldi - Before you go,123


In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

songs_chart = songs\
          .groupBy('song_name')\
          .count()\
          .orderBy('count', ascending=False)\
          .limit(10)\
          .toPandas()

fig, ax = plt.subplots(figsize=(12,12))
x = songs_chart['song_name']
y = songs_chart['count']

ax.bar(x, y)
ax.set_title('Top 10 popular songs')
plt.xticks(rotation=90)
ax.set_xlabel('Song name', wrap=True)
ax.set_ylabel('Number of plays')

for index, value in enumerate(y):
  ax.text(index, value, s=str(value), color='black', fontweight='bold', horizontalalignment='center', verticalalignment='bottom')

Most often played authors in broadcasts

In [20]:
display(songs.groupBy('author').count().orderBy("count", ascending=False).limit(20))

author,count
Ava Max,240
Sanah,219
Lady Pank,201
Viki Gabor / Kayah,191
The Weeknd,191
Dua Lipa,181
Maanam,179
Shanguy,177
Perfect,174
Alan Walker/ava Max,170


In [21]:
import matplotlib.pyplot as plt
%matplotlib inline

authors_chart = songs\
          .groupBy('author')\
          .count()\
          .orderBy('count', ascending=False)\
          .limit(10)\
          .toPandas()

fig, ax = plt.subplots(figsize=(12,12))
x = authors_chart['author']
y = authors_chart['count']

ax.bar(x, y)
ax.set_title('Top 10 popular authors')
plt.xticks(rotation=90)
ax.set_xlabel('Author', wrap=True)
ax.set_ylabel('Number of plays')

for index, value in enumerate(y):
  ax.text(index, value, s=str(value), color='black', fontweight='bold', horizontalalignment='center', verticalalignment='bottom')

Number of songs, unique songs and ratio of uniqueness played in broadcast in last 7 days

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspark.sql.functions import count, countDistinct
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter
%matplotlib inline

broadcast_chart = songs\
        .groupBy('broadcast')\
        .agg(count('song_name'), countDistinct('song_name'))\
        .withColumnRenamed('count(song_name)', 'qty_songs')\
        .withColumnRenamed('count(DISTINCT song_name)', 'qty_unique_songs')\
        .orderBy('qty_songs', ascending=False)\
        .toPandas()

width=0.45
fig, ax = plt.subplots(figsize=(12,12))
ax2 = ax.twinx()

x = broadcast_chart['broadcast']
xlen = pd.Series(np.arange(len(x)))

y1 = broadcast_chart['qty_songs']
y2 = broadcast_chart['qty_unique_songs']
y3 = y2/y1*100

bar1 = ax.bar(xlen, y1, width, color='tab:blue')
bar2 = ax.bar(xlen+width, y2, width, color='dodgerblue')
line1 = ax2.plot(xlen+width/2, y3, color='tab:cyan')

legend_elements = [Rectangle(xy=(0,0), width=0.1, height=0.2, color='tab:blue', label='Songs'),
                   Rectangle(xy=(0,0), width=0.1, height=0.2, color='dodgerblue', label='Unique songs'),
                   Line2D([0], [0], label='Percent')]

ax.legend(handles=legend_elements, loc=0)

ax.set_title('Most musicians broadcasts')
ax.set_xlabel('Broadcast')
ax.set_ylabel('Number of plays')
ax2.set_ylabel('Percent of unique songs')

ax.set_xticks(np.arange(len(x)))
ax.set_xticklabels(x)
ax2.set_ylim(0, y3.max()*1.5)
ax2.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y/100))) 


for index, value in enumerate(y1):
  ax.text(index, value, s=str(value), color='black', fontweight='bold', horizontalalignment='center', verticalalignment='bottom')
  
for index, value in enumerate(y2):
  ax.text(index+width, value, s=str(value), color='black', fontweight='bold', horizontalalignment='center', verticalalignment='bottom')
  
for index, value in enumerate(y3):
  ax2.text(index+width/2, value, s=str('{0:.2f}'.format(value)), color='black', fontweight='bold', horizontalalignment='center', verticalalignment='bottom')

Most universal authors between broadcasts

In [25]:
from pyspark.sql.functions import count, countDistinct, col

authors_broadcast = songs\
                .groupBy('author')\
                .pivot('broadcast')\
                .agg(count('*'))\
                .na.fill(0)\
                .withColumn('all',col('antyradio')+col('eska')+col('rmf-fm')+col('rock-radio')+col('trojka')+col('vox-fm')+col('zet')+col('zlote-przeboje'))\
                .orderBy('all', ascending=False)\
                .limit(36)

top_authors_broadcast_list = authors_broadcast.select('author').rdd.flatMap(lambda x: x).collect()
display(authors_broadcast.limit(20))

author,antyradio,eska,rmf-fm,rock-radio,trojka,vox-fm,zet,zlote-przeboje,all
Ava Max,0,108,57,0,0,12,63,0,240
Sanah,0,131,49,0,1,0,38,0,219
Lady Pank,31,1,22,68,2,4,18,55,201
Viki Gabor / Kayah,0,68,22,0,0,49,52,0,191
The Weeknd,0,80,54,0,0,0,57,0,191
Dua Lipa,0,92,40,0,0,1,48,0,181
Maanam,30,0,17,53,14,0,15,50,179
Shanguy,0,53,23,0,0,40,61,0,177
Perfect,42,0,20,63,5,0,10,34,174
Alan Walker/ava Max,0,90,52,0,0,0,28,0,170


In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import Window
from pyspark.sql.functions import count, sum
%matplotlib inline

overAuthor = Window.partitionBy('author')
fig, ax = plt.subplots(figsize=(12,12))

authors_broadcast = songs\
                .groupBy('author', 'broadcast')\
                .agg(count('*').alias('count'))\
                .where(col('author').isin(top_authors_broadcast_list))

authors_broadcast = authors_broadcast\
                .withColumn('count_all', sum('count').over(overAuthor))\
                .orderBy('count_all', ascending=False)

df = authors_broadcast.toPandas()
result = df.pivot(index='author', columns='broadcast', values='count').fillna(0)
result = result.astype(float)
sns.heatmap(result, annot=True, fmt='g', cmap='YlGnBu')

ax.set_title('Most universal authors')
ax.set_xlabel('Broadcast')
ax.set_ylabel('Author')


Most universal songs between broadcasts

In [28]:
from pyspark.sql.functions import count, countDistinct, col

songs_broadcast = songs\
                .groupBy('song_name')\
                .pivot('broadcast')\
                .agg(count('*'))\
                .na.fill(0)\
                .withColumn('all',col('antyradio')+col('eska')+col('rmf-fm')+col('rock-radio')+col('trojka')+col('vox-fm')+col('zet')+col('zlote-przeboje'))\
                .orderBy('all', ascending=False)\
                .limit(30)

top_songs_broadcast_list = songs_broadcast.select('song_name').rdd.flatMap(lambda x: x).collect()
display(songs_broadcast.limit(20))

song_name,antyradio,eska,rmf-fm,rock-radio,trojka,vox-fm,zet,zlote-przeboje,all
Sanah - Szampan,0,126,49,0,0,0,38,0,213
Ava Max - Salt,0,89,57,0,0,5,55,0,206
Viki Gabor / Kayah - Ramie w ramie,0,68,22,0,0,49,52,0,191
The Weeknd - Blinding lights,0,74,54,0,0,0,56,0,184
Alan Walker/ava Max - Alone pt. ii,0,90,52,0,0,0,28,0,170
Felix Jaehn‚ Vize‚ Miss Li - Close your eyes,0,95,0,0,0,12,53,0,160
Topic‚ A7S - Breaking me,0,69,42,0,0,0,45,0,156
Dua Lipa - Physical,0,73,33,0,0,0,40,0,146
Patryk Skoczyński - Dom,0,64,30,0,0,0,32,0,126
Lewis Capaldi - Before you go,0,66,48,0,0,0,9,0,123


In [29]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import Window
from pyspark.sql.functions import count, sum
%matplotlib inline

overSongName = Window.partitionBy('song_name')
fig, ax = plt.subplots(figsize=(12,12))

songs_broadcast = songs\
                .groupBy('song_name', 'broadcast')\
                .agg(count('*').alias('count'))\
                .where(col('song_name').isin(top_songs_broadcast_list))

songs_broadcast = songs_broadcast\
                .withColumn('count_all', sum('count').over(overSongName))\
                .orderBy('count_all', ascending=False)

df = songs_broadcast.toPandas()
result = df.pivot(index='song_name', columns='broadcast', values='count').fillna(0)
result = result.astype(float)
sns.heatmap(result, annot=True, fmt='g', cmap='YlGnBu')
ax.set_title('Most universal songs')
ax.set_xlabel('Broadcast')
ax.set_ylabel('Song')

Top songs in Trojka

In [31]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count, countDistinct, col, sum
from pyspark.sql import Window
%matplotlib inline

songs_trojka = songs\
                .filter(col('broadcast')=='trojka')\
                .groupBy('song_name','broadcast')\
                .agg(count('*').alias('count'))\
                .orderBy('count', ascending=False)\
                .na.fill(0)\
                .limit(30)

fig, ax = plt.subplots(figsize=(12,12))

df = songs_trojka.toPandas()
result = df.pivot(index='song_name', columns='broadcast', values='count').fillna(0)
result = result.astype(float)
sns.heatmap(result, annot=True, fmt='g', cmap='YlGnBu')
ax.set_title('Top 30 songs in Trojka')
ax.set_xlabel('Broadcast')
ax.set_ylabel('Song')

Top songs in Vox-FM

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count, countDistinct, col, sum
from pyspark.sql import Window
%matplotlib inline

songs_vox = songs\
                .filter(col('broadcast')=='vox-fm')\
                .groupBy('song_name','broadcast')\
                .agg(count('*').alias('count'))\
                .orderBy('count', ascending=False)\
                .na.fill(0)\
                .limit(30)

fig, ax = plt.subplots(figsize=(12,12))

df = songs_vox.toPandas()
result = df.pivot(index='song_name', columns='broadcast', values='count').fillna(0)
result = result.astype(float)
sns.heatmap(result, annot=True, fmt='g', cmap='YlGnBu')
ax.set_title('Top 30 songs in Vox-FM')
ax.set_xlabel('Broadcast')
ax.set_ylabel('Song')

Broadcast time - when broadcast play music in Friday 13th

In [35]:
import pandas as pd
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count, countDistinct, col, sum
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter

%matplotlib inline


df = songs\
       .filter(col('day')=='13')\
       .groupBy('broadcast', 'year', 'month', 'day', 'hour')\
       .agg(count('song_name').alias('count'))\
       .toPandas()

df['datetime'] = (pd.to_datetime(df['year'].astype(str) + '-' +
                                  df['month'].astype(str) + '-' +
                                  df['day'].astype(str) + ' ' +
                                  df['hour'].astype(str) + ':00:00'))

df.sort_values(by='datetime', inplace=False)

fig, ax = plt.subplots(figsize=(12,12))
ax = sns.lineplot(x='datetime', y='count', hue='broadcast', data=df)
ax.legend(loc=0)
ax.set_title('Broadcasts time')
ax.set_xlabel('Datetime')
ax.set_ylabel('Played songs')

Detailed comparision broadcast time in Trojka and Rmf-FM

In [37]:
import pandas as pd
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count, countDistinct, col, sum
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter
%matplotlib inline


df = songs\
       .filter(col('broadcast').isin(['trojka', 'rmf-fm']))\
       .groupBy('broadcast', 'year', 'month', 'day', 'hour')\
       .agg(count('song_name').alias('count'))\
       .toPandas()

df['datetime'] = (pd.to_datetime(df['year'].astype(str) + '-' +
                                  df['month'].astype(str) + '-' +
                                  df['day'].astype(str) + ' ' +
                                  df['hour'].astype(str) + ':00:00'))

df.sort_values(by='datetime', inplace=False)

fig, ax = plt.subplots(figsize=(12,12))
ax = sns.lineplot(x='datetime', y='count', hue='broadcast', data=df)
ax.set_title('Broadcasts time')
ax.set_xlabel('Datetime')
ax.set_ylabel('Played songs')

Detailed view for Saturday 14th March in Trojka, Rmf-FM, Eska, Zet

In [39]:
import pandas as pd
import matplotlib.dates as mdates
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import count, countDistinct, col, sum
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter
%matplotlib inline


df = songs\
       .filter((col('broadcast').isin(['trojka', 'rmf-fm', 'eska', 'zet'])) & (col('day')=='14')) \
       .groupBy('broadcast', 'year', 'month', 'day', 'hour')\
       .agg(count('song_name').alias('count'))\
       .toPandas()

df['datetime'] = (pd.to_datetime(df['year'].astype(str) + '-' +
                                  df['month'].astype(str) + '-' +
                                  df['day'].astype(str) + ' ' +
                                  df['hour'].astype(str) + ':00:00'))

df.sort_values(by='datetime', inplace=False)

fig, ax = plt.subplots(figsize=(12,12))
ax = sns.lineplot(x='datetime', y='count', hue='broadcast', data=df)
ax.set_title('Broadcasts time')
ax.set_xlabel('Datetime')
ax.set_ylabel('Played songs')
